In [1]:
import pymongo
from flask import Flask
from pymongo.database import Database
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

C:\Users\10309\AppData\Local\Temp\ipykernel_16528\1067504827.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def mongodb_init():
    # connect to mongodb
    mongo = pymongo.MongoClient(host='18.162.214.19', port=27017, username="root", password="1647#4hkust",
                                authSource='admin')
    print('数据库当前的databases: ', mongo.list_database_names())
    return mongo


def get_db(mongo, db_name):
    db = Database(name=db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db


# API below
app = Flask(__name__)
mongo = mongodb_init()
db = get_db(mongo, 'chen_db')

数据库当前的databases:  ['admin', 'chen_db', 'config', 'local']
获取/创建库： chen_db


In [3]:
# data = {"PID":[],"Title":[],"Text":[]}
# Posts = db.Posts.aggregate([ {'$match': {'Deleted': {'$ne': True}}}])
# # Posts = db.Posts.find()
# if Posts:
#     for Post in Posts:
#         data["PID"].append(str(Post['_id']))
#         data["Title"].append(Post.get("Title").strip("\n"))
#         data["Text"].append(Post.get("Text").strip("\n"))
        
# df = pd.DataFrame(data)
# print(df)

In [4]:
data = []
Posts = db.Posts.aggregate([ {'$match': {'Deleted': {'$ne': True}}} ])
# Posts = db.Posts.find()
if Posts:
    for Post in Posts:
        data.append([str(Post['_id']), Post.get("Title").strip("\n"), Post.get("Text").strip("\n")])
df = pd.DataFrame(data, columns=['PID', 'Title', 'Text'])
print(df["Text"])

0     These ExploreOne 4K Wi-Fi Action Camera Kits w...
1     Used Sony 600mm F4 lens in immaculate conditio...
2     For sale is a fully custom BMX bike with all t...
3     Price is negotiable (price already dropped 250...
4     Used once or twice good condition. Price negot...
5     As new in excellent condition.\nPremium heavy ...
6     LULULEMON MENS commission pants\nGreat conditi...
7     These are brand new, never worn (feel free to ...
8     solid wood desk sale\n\nNew Mattresses sale: S...
9           X Games BMX, fully serviced, 18 inch wheels
10    Item is for vintage techniques. Keyboard Orego...
11    Burswood Bass Guitar fairly small Bass electri...
12    I have a nice , light, powerful, small Fender ...
13    Epiphone classical guitar for fingerstyle play...
14    Used only a few times, great condition. Delive...
15    Up for grabs is pair of CCM Jetspeed FT6Pro ho...
16    Like new HEAD squash racket:\n- forgiving head...
17    Hello,\nI’m selling a good condition helme

In [5]:
import re
import spacy
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords

class Utils():
    def __init__(self):
        """ Class Constructor """
        self.stop_words = stopwords.words('english')
        unwanted_stopwords = {'no', 'nor', 'not', 'ain', 'aren', "aren't", 'couldn', 'what', 'which', 'who',
                                      'whom',
                                      'why', 'how', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't",
                                      'hasn',
                                      "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
                                      "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
                                      "wasn't",
                                      'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'don', "don't"}

        self.stop_words = [ele for ele in self.stop_words if ele not in unwanted_stopwords]
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.nouns = ['NNP', 'NNPS']
        self.nlp = spacy.load('en_core_web_sm')
        self.label_encoder = LabelEncoder()
        
    def clean_text(self, text, remove_stopwords=True, lemmatize=True):
            """ Function to clean text
            @param text (str): text to be cleaned
            @param remove_stopwords (bool): To remove stopwords or not.
            @param lemmatize (bool): to lemmatize or not.
            """

            # Remove emails 
            text = re.sub('\S*@\S*\s?', '', text)
            # Remove new line characters 
            text = re.sub('\s+', ' ', text) 
            # Remove distracting single quotes 
            text = re.sub("\'", '', text)
            # Remove puntuations and numbers
            text = re.sub('[^a-zA-Z]', ' ', text)
            # Remove single characters
            text = re.sub('\s+[a-zA-Z]\s+^I', ' ', text)
            # Remove accented words
            text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            # remove multiple spaces
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'^\s*|\s\s*', ' ', text).strip()
            text = text.lower()

            if not remove_stopwords and not lemmatize:
                return text

            # Remove unncecessay stopwords
            if remove_stopwords:
                text = word_tokenize(text)
                text = " ".join([word for word in text if word not in self.stop_words])
            
            # Word lemmatization
            if lemmatize:
                text = self.nlp(text)
                lemmatized_text = []
                for word in text:
                    if word.lemma_.isalpha():
                        if word.lemma_ != '-PRON-':
                            lemmatized_text.append(word.lemma_.lower())
                text = " ".join([word.lower() for word in lemmatized_text])
                    
            return text
        
    def prepare_data(self, df, remove_stopwords=True, lemmatize=True):
        df = df.sample(frac=1).reset_index(drop=True)
        x_train = df['Text'].apply(lambda x: self.clean_text(str(x), remove_stopwords=remove_stopwords, lemmatize=lemmatize))
        return x_train

In [6]:
utils = Utils()

In [7]:
prepare_data = utils.prepare_data(df, remove_stopwords=True, lemmatize=True)
print("Total training examples: ", len(prepare_data))

Total training examples:  18


In [18]:
# 使用TF-IDF向量化商品标题和描述
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(prepare_data)
print(tfidf_matrix)

  (0, 356)	0.045932832434116916
  (0, 226)	0.045932832434116916
  (0, 117)	0.04020459823606808
  (0, 318)	0.09186566486823383
  (0, 439)	0.045932832434116916
  (0, 102)	0.045932832434116916
  (0, 349)	0.03298787662727461
  (0, 233)	0.04020459823606808
  (0, 196)	0.03298787662727461
  (0, 96)	0.045932832434116916
  (0, 227)	0.045932832434116916
  (0, 370)	0.045932832434116916
  (0, 242)	0.04020459823606808
  (0, 142)	0.04020459823606808
  (0, 452)	0.045932832434116916
  (0, 27)	0.045932832434116916
  (0, 262)	0.045932832434116916
  (0, 300)	0.045932832434116916
  (0, 373)	0.045932832434116916
  (0, 313)	0.09186566486823383
  (0, 141)	0.09186566486823383
  (0, 267)	0.045932832434116916
  (0, 58)	0.045932832434116916
  (0, 285)	0.045932832434116916
  (0, 60)	0.036140351393046895
  :	:
  (16, 413)	0.18641924274232954
  (16, 170)	0.18641924274232954
  (16, 390)	0.18641924274232954
  (16, 31)	0.18641924274232954
  (16, 175)	0.18641924274232954
  (16, 252)	0.18641924274232954
  (16, 329)	0.37

In [8]:
from gensim.models import Word2Vec
import numpy as np

# 训练 Word2Vec 模型
model = Word2Vec(prepare_data, vector_size=100, window=5, min_count=1, sg=0)
# 获取单词的向量表示
word_vectors = model.wv

# 生成文档向量
document_vectors = []
for doc in prepare_data:
    doc_vector = np.mean([word_vectors[word] for word in doc], axis=0)
    document_vectors.append(doc_vector)
print(document_vectors)

[array([-0.11391317,  0.22143367,  0.20598006,  0.08948959,  0.00905665,
       -0.22953895,  0.24389428,  0.5320412 , -0.33306047, -0.4027382 ,
        0.12868506, -0.32460645, -0.06159396,  0.10010482,  0.06574864,
       -0.05889767,  0.35044327,  0.15094426, -0.13936631, -0.6513894 ,
        0.10913548, -0.00663832,  0.47811475, -0.03744963, -0.07763944,
        0.18973121, -0.21295606,  0.09380717, -0.07001911,  0.11358386,
        0.22209811, -0.16305807,  0.10563266, -0.27379006, -0.07103338,
        0.1677737 ,  0.19735134, -0.01186027, -0.14177455, -0.08161841,
        0.25656107, -0.19084802, -0.23205231,  0.19517615,  0.07895064,
       -0.06695022, -0.21614286, -0.16195431,  0.03819529,  0.08285403,
        0.09910321, -0.247823  , -0.12295167, -0.05725861, -0.15900294,
       -0.11170746,  0.13196619, -0.20291612,  0.01183293,  0.04500766,
       -0.17253628, -0.14107904,  0.413506  ,  0.02109236, -0.1727915 ,
        0.42760178,  0.05941591,  0.24414241, -0.29004744,  0.0

In [12]:
# 使用K-Means聚类算法将商品分为3个簇
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters)
# kmeans.fit(tfidf_matrix)
kmeans.fit(document_vectors)

d:\Anaconda3\envs\ECE_651\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=5)

In [13]:
# 将聚类结果添加到数据集中
df['cluster'] = kmeans.labels_

In [14]:
# 打印每个簇中的商品
for cluster_id in range(num_clusters):
    cluster_products = df[df['cluster'] == cluster_id]['Title'].tolist()
    print(f"Cluster {cluster_id + 1}: {', '.join(cluster_products)}")

Cluster 1: ExploreOne 4K Wi-Fi Action Camera with 32 GB Micro SD - $32.00., Sony 14mm F1.8 GM., Custom Fiend/Cult BMX Build (MUST SEE)., LULULEMON commission pants MENS large -black., guitar amp.
Cluster 2: Custom BMX for sale., Vintage techniques, keyboard,organ.
Cluster 3: Men's Jacket. Size Small., Ray-Ban RB3671CH Chromance Sunglasses., solid wood desk sale., X Games 18 inch BMX., Burswood Bass Guitar $120 O.B.O., Boys Black Hespeler Jr Hockey Skate Shoes, US J3. $25 OBO., HEAD Graphene XT Cyano 120 Squash Racket.
Cluster 4: Epiphone classical fingerstyle guitar., CCM Jetspeed FT6Pro Hockey skates size 8 regular., Bauer Girls Helmet.
Cluster 5: shadow bike pedals and grips.
